| Method    | Description | SQL equivalents |
| -------- | ------- | ------- |
|`GroupBy` |Groups a sequence into subsequences |GROUP BY
|`Chunk` |Groups a sequence into arrays of a fixed size|

### GroupBy

`IEnumerable<TSource>`→`IEnumerable<IGrouping<TKey,TElement>>`  
  
| Argument    | Type |
| -------- | ------- |
|***Input sequence*** |`IEnumerable<TSource>`
|***Key selector*** |`TSource => TKey`
|***Element selector*** (optional) | `TSource => TElement`
|`Comparer` (optional) | `IEqualityComparer<TKey>`

***GroupBy*** organizes a `flat input sequence` into `sequences of groups`.

In [ ]:
using System.IO;
string[] files = Directory.GetFiles (Path.GetTempPath());

IEnumerable<IGrouping<string,string>> query =
    files.GroupBy (file => Path.GetExtension (file));

foreach (IGrouping<string,string> grouping in query)
{
    Console.WriteLine ("Extension: " + grouping.Key);
    foreach (string filename in grouping)
        Console.WriteLine (" - " + filename);
}

/*
Extension: .pdf
-- chapter03.pdf
-- chapter04.pdf
Extension: .doc
-- todo.doc
-- menu.doc
-- Copy of menu.doc
...
*/

In [ ]:
using System.IO;
string[] files = Directory.GetFiles (Path.GetTempPath());

//element selector
var query =
    files.GroupBy (file => Path.GetExtension (file), file=> file.ToUpper());

//orderby
var query1 =
    files.GroupBy (file => Path.GetExtension (file), file=> file.ToUpper())
         .OrderBy(grouping=> grouping.Key);

In [ ]:
using System.IO;
string[] files = Directory.GetFiles (Path.GetTempPath());

//query syntax
var query =
    from file in files
    group file by Path.GetExtension(file) into grouping
    select grouping;

//element selector
var query2 =
    from file in files
    group file.ToUpper() by Path.GetExtension(file) into grouping
    select grouping;

//orderby
var query3 =
    from file in files
    group file.ToUpper() by Path.GetExtension(file) into grouping
    orderby grouping.Key
    select grouping;

//with filters
var query4 =
    from file in files
    group file.ToUpper() by Path.GetExtension(file) into grouping
    where grouping.Count() >= 5
    select grouping;

/*A where after a group by is equivalent to HAVING in SQL.*/

//sort by count

var query5 =
    from file in files
    group file by Path.GetExtension(file) into grouping
    orderby grouping.Count() descending
    select grouping;

### GroupBy in EF Core

In [ ]:
var query = 
    from p in dbContext.Purchases
    group p.Price by p.Date.Year into salesByYear
    select new 
        {
            Year = salesByYear.Key,
            TotalValue = salesByYear.Sum()
        };

### Grouping by multiple keys

In [4]:
var names = new List<string> { "Jay", "Tom", "Mary", "Dick", "Harry", "Jun", "Dock" };

var query =
    from n in names
    group n by new { FirstLetter = n[0], Length = n.Length };
foreach(var grouping in query)
{
    Console.WriteLine(grouping.Key.FirstLetter +" : " + grouping.Key.Length);
    foreach(var item in grouping )
    Console.WriteLine("--"+ item);
}

J : 3
--Jay
--Jun
T : 3
--Tom
M : 4
--Mary
D : 4
--Dick
--Dock
H : 5
--Harry
j : 3
--jim


### Custom equality comparers

in a **local query**, You can pass a `custom equality comparer` into ***GroupBy***, to `change` the `algorithm` for `key comparison`.

In [7]:
var names = new List<string> { "Jay", "Mary", "Harry", "jun" };

var query =
    from n in names
    group n by n[0] ;
foreach(var grouping in query)
{
    Console.WriteLine(grouping.Key);
    foreach(var item in grouping )
    Console.WriteLine("--"+ item);
}

Console.WriteLine("*********************************************");

var query2 =
    from n in names
    group n by char.ToUpper(n[0]);
foreach(var grouping in query2)
{
    Console.WriteLine(grouping.Key);
    foreach(var item in grouping )
    Console.WriteLine("--"+ item);
}

J
--Jay
M
--Mary
H
--Harry
j
--jun
*********************************************
J
--Jay
--jun
M
--Mary
H
--Harry


### Chunk

`IEnumerable<TSource>`→`IEnumerable<TElement[]>`

| Argument    | Type |
| -------- | ------- |
|***Input sequence*** |`IEnumerable<TSource>`
|***size*** |`int`

***Chunk*** `groups a sequence` into `chunks of a given size`

In [8]:
foreach (int[] chunk in new[] { 1, 2, 3, 4, 5, 6, 7, 8 }.Chunk (3))
Console.WriteLine (string.Join (", ", chunk));

1, 2, 3
4, 5, 6
7, 8
